In [8]:
# 1.Daily covid cases

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime
df=pd.read_excel(r'DataSource/airports.xlsx')
df=df.drop(columns=['Unit'])
df.to_hdf('./Processed_Data/airports.h5',key='raw')

ap=pd.read_hdf('./Processed_Data/airports.h5',key='raw')
ap=ap.groupby(by='城市').sum().reset_index()
lastcol=ap.columns[-1]
ap=ap.sort_values(by=lastcol,ascending=False).head(50)
ap=pd.melt(ap,id_vars=['城市'],var_name='date',value_name='air_traffic')
ap=ap[ap['air_traffic']>0]

ap['date'] = pd.to_datetime(ap['date']).dt.date
ap=ap.sort_values(by=['城市','date'],ascending=True)
ap['date']=ap['date'].astype('str')

def getdt(d):
    return str(d)[-5:]
ap['dt']=ap['date'].apply(getdt)

ap['ap_last7d_mean']=ap.groupby('城市')['air_traffic'].transform(lambda x: x.rolling(7, 1).mean())
ap['ap_WoW']=ap['ap_last7d_mean']/ap.groupby('城市')['ap_last7d_mean'].transform(lambda x: x.shift(periods=7)) -1

c1=ap['date'].str.contains('2019')
ap_19=ap[c1]
ap_19=ap_19[['城市', 'dt','ap_last7d_mean']]
ap_19=ap_19.rename(columns={'ap_last7d_mean':'ap_19_rolling_mean'})

c2=ap['date'].str.contains('2022|2023')
ap_22=ap[c2]
ap_22=ap_22.merge(ap_19,how='left',on=['城市','dt'])

ap_22['ap_status']=ap_22['ap_last7d_mean']/ap_22['ap_19_rolling_mean']

ap_22.tail(3)

,城市,date,air_traffic,dt,ap_last7d_mean,ap_WoW,ap_19_rolling_mean,ap_status
18130,青岛,2022-12-31,204.0,12-31,175.714286,-0.129512,484.285714,0.362832
18131,青岛,2023-01-01,199.0,01-01,181.428571,-0.046547,451.000000,0.402281
18132,青岛,2023-01-02,208.0,01-02,188.571429,0.046788,455.000000,0.414443


In [9]:
# TY 机场 vs. 2019
%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
riqi='2023-01-02'

df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
o=ap_22.merge(df_covid,on=['城市'],how='left')

o=o[o['ap_status']<2][o['date']==riqi][['城市','progress_2','ap_status']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)
o.to_clipboard(index=False,header=None)

d=ap_22[ap_22['date']==riqi][['城市','date','ap_status']].reset_index(drop=True).rename(columns={'ap_status':'airports'})
d['date']=pd.to_datetime(d['date'])
d.to_hdf('Processed_Data/city_compare.h5',key='airports')

print(riqi)
print(o)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2023-01-02
         城市  progress_2  ap_status  size group
9433     昆明         4.0   0.538073   NaN  较早达峰
15248    西安         1.0   0.412268   NaN  较早达峰
11242    泸州         1.0   1.503759   NaN  较早达峰
11607    济南         3.0   0.529304   NaN  较早达峰
7972     成都         NaN   0.930310   NaN  较早达峰
13798   石家庄         2.0   0.391600   NaN  较早达峰
6145     天津         2.0   0.371692   NaN  较早达峰
5780     大连         5.0   0.393263   NaN  较早达峰
10528    沈阳         2.0   0.569782   NaN  较早达峰
14890    西宁         5.0   0.544031   NaN  较早达峰
10163    武汉         NaN   0.490985   NaN  较早达峰
1825     兰州         3.0   0.573648   NaN  较早达峰
16344    郑州         NaN   0.475806   NaN  较早达峰
16709    重庆         0.0   0.520342   NaN  较早达峰
2190     北京         2.0   0.458061   NaN  较早达峰
17402    长春         4.0   0.688081   NaN  较早达峰
15612    贵阳         4.0   0.612245   NaN    其他
10876    泉州        20.0   0.439909   NaN    其他
17767   

In [10]:
# TY 机场数据 WoW
# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
o=ap_22.merge(df_covid,on=['城市'],how='left')

o=o[o['date']==riqi][['城市','progress_2','ap_WoW']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-02


In [11]:
# TY 机场数据 WoW vs. GDP
# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
o=ap_22.merge(city_popu,on=['城市'],how='left')

o=o[o['date']==riqi][['城市','GDP','ap_WoW']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-02


In [15]:
# TY 机场数据 as of 19 vs. GDP
# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
o=ap_22.merge(city_popu,on=['城市'],how='left')

o=o[o['date']==riqi][o['ap_status']<1.5][['城市','GDP','ap_status']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='较早达峰'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-02


In [12]:
citylist=['北京','石家庄','成都','重庆','武汉','郑州']
c3=ap_22['城市'].isin(citylist)
c4=ap_22['date']>'2022-03-01'
show=ap_22[c3&c4]


show=pd.pivot_table(show,index='date',columns='城市',values='ap_status',aggfunc='mean').reset_index()
show['sum']=np.nan

show=show[['date','sum']+citylist]
show.to_clipboard(index=False)